# 提取每条链的所有氨基酸的特征

如：cb513的第一条链的第一个氨基酸

cb513_cha0_amni0&V&9|3|2|2.5|-0.523|0.10365945641690333|-0.18643152963488993|-0.47042481056671664&0.2141650169574414|0.17079548202237446|0.022977369910025615|0.04653047451473147|0.8641271029909058|0.02231343869903224|0.13354172253321245|0.8869541699279211|0.0623858513799944|0.9248398905178734|0.7310585786300049|0.03356922328148252|0.042289771842033794|0.07516010948212662|0.06476696860678553|0.08548913943480654|0.2573094546973142|0.9234378026481879|0.27091207765069353|0.2689414213699951|0.9234378026481879|0.0

In [24]:
import numpy as np
import gzip
from math import log

#氨基酸列表
Amino=['A', 'C', 'E', 'D', 'G', 'F', 'I', 'H', 'K', 'M', 'L', 'N', 'Q', 'P', 'S', 'R', 'T', 'W', 'V', 'Y', 'A']

# 氨基酸的3个非数值特征
Sidechainclass=sorted(['acid','aliphatic','amide','aromatic','basic','basic-aromatic','cyclic','hydroxyl-containing','sulfur-containing'])
Sidechainpola=sorted(['acidic-polar','basic-polar','nonpolar','polar'])
Sidechainchar=sorted(['neutral','positive','negative'])

# 把positive(10%)neutral(90%)当做一个新的特征Sidechainchar
Sidechainchartmp=sorted(['neutral','positive','negative','positive(10%)neutral(90%)'])

#二级结构标签
SSLabletmp=['L', 'B', 'E', 'G', 'I', 'H', 'S', 'T']
SSLable={'C':['L','S','T'],'E':['B','E'],'H':['G','I','H']}


# 读取氨基酸6维属性，非6大类，每个氨基酸有6维特征，
aminoacid={}
for line in open('aminoacid'):
    line=line.replace('\n','')
    line=line.split()
    # print(line,len(line))
    aminoacid[line[1].upper()]=line[2:6]
    aminoacid[line[1].upper()].extend(line[8:10])

In [25]:
def GetSSlable(sslabletmp):
    if sslabletmp in SSLable['C']:
        return 'C'
    elif sslabletmp in SSLable['E']:
        return 'E'
    elif sslabletmp in SSLable['H']:
        return 'H'
    else:
        return 'None'

In [26]:
#读取数据
def load_gz(filepath):  
    f=gzip.open(filepath,'rb')
    
    return np.load(f)

In [27]:
#分析某一条链上的氨基酸

def GetAminInfo(chaininfo,chainname):
    amininfo={}
    
    for poiind in range(len(chaininfo)):
        aminind=(chaininfo[poiind][:22].tolist()).index(1)
        if aminind>20:
            continue
        else:
            aminname=Amino[aminind]
            sslabel=GetSSlable(SSLabletmp[(chaininfo[poiind][22:31].tolist()).index(1)])
            pssm=(chaininfo[poiind][35:57]).tolist()
        
        amininfo[chainname+'_amni%s'%poiind]=[aminname,sslabel,pssm]
    return amininfo

In [28]:
#统计每类氨基酸在某一二级结构下出现的次数

def CountSSlabel(chainaminsinfo):
    
    Conutsslabletmp={}
    
    for amininfo in sorted(chainaminsinfo):
        amin=chainaminsinfo[amininfo][0]
        sslabel=chainaminsinfo[amininfo][1]
        
        if sslabel in Conutsslabletmp:
            if amin in Conutsslabletmp[sslabel]:
                Conutsslabletmp[sslabel][amin]+=1
            else:
                Conutsslabletmp[sslabel][amin]=1
        else:
            Conutsslabletmp[sslabel]={}
            Conutsslabletmp[sslabel][amin]=1
    return Conutsslabletmp

In [29]:
# 获取某个氨基酸的8维特征

def GetAminFea(amin,Conutallsslable):
    aminfea=[]
    
    #前面5维特征
    for feaind in range(len(aminoacid[amin])-1):
        if feaind==0:
            aminfea.append(Sidechainclass.index(aminoacid[amin][feaind])+1)
        if feaind==1:
            aminfea.append(Sidechainpola.index(aminoacid[amin][feaind])+1)
        if feaind==2:
            aminfea.append(Sidechainchartmp.index(aminoacid[amin][feaind])+1)
        if feaind>2:
            aminfea.append(aminoacid[amin][feaind])
    #后面三维统计特征
    for sslable in SSLable:
        rates=log(float(aminoacid[amin][-1])*1.0/(Conutallsslable[sslable][amin]*100.0/sum(Conutallsslable[sslable].values())))
        aminfea.append(rates)
    return aminfea

In [30]:
#_____**没有使用**_______统计所有文件中每类氨基酸在某一二级结构下出现的次数
def CountAllSSlabel():
    Conutsslable={}
    
    cb513=load_gz('cb513.npy.gz')
    cb513=cb513.reshape(cb513.shape[0],700,57)
    #跑一遍，处理统计信息
    for aminind  in range(len(cb513)):
        #分析某一条链上的氨基酸
        chainaminsinfo=GetAminInfo(cb513[aminind],'%s_cha%s'%('cb513.npy.gz',aminind))

        #统计每类氨基酸在某一二级结构下出现的次数
        Conutsslable['%s_cha%s'%('cb513.npy.gz',aminind)]=CountSSlabel(chainaminsinfo,Conutsslable)

    cullpdb=load_gz('cullpdb.npy.gz')
    cullpdb=cullpdb.reshape(cullpdb.shape[0],700,57)
    #跑一遍，处理统计信息
    for aminind  in range(len(cullpdb)):
        #分析某一条链上的氨基酸
        chainaminsinfo=GetAminInfo(cullpdb[aminind],'%s_cha%s'%('cullpdb.npy.gz',aminind))

        #统计每类氨基酸在某一二级结构下出现的次数
        Conutsslable['%s_cha%s'%('cb513.npy.gz',aminind)]=CountSSlabel(chainaminsinfo,Conutsslable)
    
    return Conutsslable

In [31]:
def GetAlllSSlabel(filename):
    filenametmp='Oridata/'+filename+'.npy.gz'
    cb513=load_gz(filenametmp)
    cb513=cb513.reshape(cb513.shape[0],700,57)
    
    Conutallsslable={}
    #先跑一遍，处理统计信息
    for aminind  in range(len(cb513)):
        #最后一条链取倒数54个
        if 'cb513' in filename and aminind==len(cb513)-1:
            calaminInfo=cb513[aminind][-54:]
        else:
            calaminInfo=cb513[aminind]
        
        #分析某一条链上的氨基酸
        chainaminsinfo=GetAminInfo(calaminInfo,'%s_cha%s'%(filename,aminind))

        #统计每类氨基酸在某一二级结构下出现的次数
        Conutallsslabletmp=CountSSlabel(chainaminsinfo)
        
        #更新每类氨基酸在某一二级结构下出现的次数
        for sslable in SSLable:
            for amino in set(Amino):
                if sslable in Conutallsslabletmp and amino in Conutallsslabletmp[sslable]:
                    if sslable in Conutallsslable:
                        if amino in Conutallsslable[sslable]:
                            Conutallsslable[sslable][amino]+=Conutallsslabletmp[sslable][amino]
                        else:
                            #Conutallsslable[sslable]={}
                            Conutallsslable[sslable][amino]=Conutallsslabletmp[sslable][amino]
                    else:
                        Conutallsslable[sslable]={}
                        Conutallsslable[sslable][amino]=Conutallsslabletmp[sslable][amino]
    return Conutallsslable

In [32]:
#构建某个文件的所有链的所有氨基酸特征
def BuildFea(filename,Conutallsslable):
    
    filenametmp='Oridata/'+filename+'.npy.gz'
    cb513=load_gz(filenametmp)
    cb513=cb513.reshape(cb513.shape[0],700,57)

    fw_fea=open('0.Featrue/aminsinfo_%s_Q3'%filename,'w')
    fw_list=open('0.Featrue/aminslist_%s_Q3'%filename,'w')
    
#     Conutallsslable={}
#     #先跑一遍，处理统计信息
#     for aminind  in range(len(cb513)):
#         #最后一条链取倒数54个
#         if 'cb513' in filename and aminind==len(cb513)-1:
#             calaminInfo=cb513[aminind][-54:]
#         else:
#             calaminInfo=cb513[aminind]
        
#         #分析某一条链上的氨基酸
#         chainaminsinfo=GetAminInfo(calaminInfo,'%s_cha%s'%(filename,aminind))

#         #统计每类氨基酸在某一二级结构下出现的次数
#         Conutallsslabletmp=CountSSlabel(chainaminsinfo)
        
#         #更新每类氨基酸在某一二级结构下出现的次数
#         for sslable in SSLable:
#             for amino in set(Amino):
#                 if sslable in Conutallsslabletmp and amino in Conutallsslabletmp[sslable]:
#                     if sslable in Conutallsslable:
#                         if amino in Conutallsslable[sslable]:
#                             Conutallsslable[sslable][amino]+=Conutallsslabletmp[sslable][amino]
#                         else:
#                             #Conutallsslable[sslable]={}
#                             Conutallsslable[sslable][amino]=Conutallsslabletmp[sslable][amino]
#                     else:
#                         Conutallsslable[sslable]={}
#                         Conutallsslable[sslable][amino]=Conutallsslabletmp[sslable][amino]
        

    #再跑一遍，存储特征
    for aminind  in range(len(cb513)):
        
        #最后一条链取倒数54个
        if 'cb513' in filename and aminind==len(cb513)-1:
            calaminInfo=cb513[aminind][-54:]
        else:
            calaminInfo=cb513[aminind]       
        
        #分析某一条链上的氨基酸
        chainaminsinfo=GetAminInfo(calaminInfo,'%s_cha%s'%(filename,aminind))

        #存储到文件上
        for amininfo in sorted(chainaminsinfo):
            #氨基酸本身特征
            aminfea=GetAminFea(chainaminsinfo[amininfo][0],Conutallsslable)
            
            allamininfo=amininfo+'&'+chainaminsinfo[amininfo][0]+'&'+chainaminsinfo[amininfo][1]+'&'+'|'.join(list(map(str,aminfea)))+'&'+'|'.join(list(map(str,chainaminsinfo[amininfo][2])))
            fw_fea.write(allamininfo+'\n')
            fw_list.write(amininfo+'&'+chainaminsinfo[amininfo][0]+'&'+chainaminsinfo[amininfo][1]+'\n')
    fw_fea.close()
    fw_list.close()
    
    print('filename,Conutallsslable: ',filename,Conutallsslable,sum(Conutallsslable['C'].values()),sum(Conutallsslable['E'].values()),sum(Conutallsslable['H'].values()))
    print(sum(Conutallsslable['C'].values())+sum(Conutallsslable['E'].values())+sum(Conutallsslable['H'].values()))

In [33]:
#Conutallsslable=CountAllSSlabel()

In [34]:
Conutallsslable=GetAlllSSlabel('cullpdb')

In [35]:
BuildFea('cb513',Conutallsslable)
BuildFea('cullpdb',Conutallsslable)

filename,Conutallsslable:  cb513 {'C': {'A': 32029, 'I': 15563, 'D': 37728, 'G': 55470, 'W': 4993, 'Y': 12691, 'S': 33905, 'R': 20321, 'T': 27255, 'N': 28515, 'E': 26189, 'L': 28614, 'V': 20120, 'C': 5439, 'M': 5900, 'P': 36914, 'Q': 14916, 'F': 14221, 'K': 25595, 'H': 11881}, 'E': {'A': 18862, 'I': 26725, 'D': 8966, 'G': 13012, 'W': 4913, 'Y': 13486, 'S': 14062, 'R': 12633, 'T': 17887, 'N': 7205, 'E': 12733, 'L': 27783, 'V': 35583, 'C': 4350, 'M': 4232, 'P': 5566, 'Q': 7661, 'F': 15701, 'K': 12261, 'H': 6288}, 'H': {'A': 55059, 'I': 26371, 'D': 23820, 'G': 16239, 'W': 6860, 'Y': 15923, 'S': 22080, 'R': 27293, 'T': 18894, 'N': 14901, 'E': 41980, 'L': 54339, 'V': 27324, 'C': 4686, 'M': 8739, 'P': 11127, 'Q': 21755, 'F': 18562, 'K': 29760, 'H': 9438}} 458259 269909 455150
1183318
filename,Conutallsslable:  cullpdb {'C': {'A': 32029, 'I': 15563, 'D': 37728, 'G': 55470, 'W': 4993, 'Y': 12691, 'S': 33905, 'R': 20321, 'T': 27255, 'N': 28515, 'E': 26189, 'L': 28614, 'V': 20120, 'C': 5439, 'M'